In [1]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.4 MB/s eta 0:

In [3]:
# Usual Libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')

In [4]:
general_path = '/content/drive/MyDrive/Data'
print(list(os.listdir(f'{general_path}/genres_original/')))

['classical', 'metal', 'hiphop', 'rock', 'pop', 'country', 'disco', 'reggae', 'blues', 'jazz']


In [55]:
import gradio as gr
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg
import sklearn.preprocessing

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)
def plot_audio_features(file,graph):
    # Load audio file
    y, sr = librosa.load(file.name)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0][0]
    tempo, _ = librosa.beat.beat_track(y=y, sr = sr)
    if (graph=="All"):
        # Compute features
        stft = np.abs(librosa.stft(y))
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        spectrogram = librosa.amplitude_to_db(stft, ref=np.max)
        mel_spec = librosa.amplitude_to_db(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000), ref=np.max)
        harmonic_percussive = librosa.effects.hpss(y)
        # Computing the time variable for visualization
        frames = range(len(spectral_centroids))

        # Converts frame counts to time (seconds)
        t = librosa.frames_to_time(frames)
        # Set up the plot
        fig, ax = plt.subplots(nrows=7, sharex=True, figsize=(8,12))

        # Plot waveform
        librosa.display.waveshow(y, sr=sr, ax=ax[0], color='#FFB100',alpha=0.4)
        ax[0].set(title='Waveform')

        # Plot STFT
        librosa.display.specshow(spectrogram, sr=sr, x_axis='time', y_axis='linear', ax=ax[1],cmap='coolwarm')
        ax[1].set(title='STFT (dB)')

        # Plot Spectrogram
        librosa.display.specshow(spectrogram, sr=sr, x_axis='time', y_axis='log', ax=ax[2],cmap='coolwarm')
        ax[2].set(title='Spectrogram (dB)')

        # Plot Mel Spectrogram
        librosa.display.specshow(mel_spec, sr=sr, x_axis='time', y_axis='mel', ax=ax[3],cmap='coolwarm')
        ax[3].set(title='Mel Spectrogram (dB)')

        # Plot Harmonics and Percussive components
        librosa.display.waveshow(harmonic_percussive[0], sr=sr, alpha=0.4, ax=ax[4], label='Harmonic',color='#FFB100')
        librosa.display.waveshow(harmonic_percussive[1], sr=sr, alpha=0.4, ax=ax[4], label='Percussive',color='#0F52BA')
        ax[4].legend(loc='upper right')
        ax[4].set(title='Harmonics and Percussive')
        #Plot the spectral centroids
        librosa.display.waveshow(y, sr=sr, alpha=0.4, color='#FFB100', ax=ax[5])
        ax[5].set(title='Waveform + Spectral centroids')
        ax[5].plot(t, normalize(spectral_centroids), color='#0F52BA',linewidth=0.7)
        #Plot the spectral RollOff
        librosa.display.waveshow(y, sr=sr, alpha=0.4, color='#FFB100', ax=ax[6])
        frames = range(len(rolloff))
        t = librosa.frames_to_time(frames)
        ax[6].set(title='Waveform + Spectral RollOff')
        ax[6].plot(t, normalize(rolloff), color='#0F52BA',linewidth=0.7)
        # Add space between subplots

        plt.subplots_adjust(hspace=0.7)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    elif (graph=="Waveform"):
        # Set up the plot
        fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(16,6))
        # Plot waveform
        librosa.display.waveshow(y, sr=sr, color='#FFB100',alpha=0.4)
        plt.title('Waveform', fontsize = 23)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    elif (graph=="Short-time Fourier Transform (STFT)"):
        # Set up the plot
        fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(16,6))
        stft = np.abs(librosa.stft(y))
        # Plot STFT
        plt.plot(stft)
        plt.title('STFT (dB)', fontsize = 23)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    elif (graph=="Spectrogram"):
        # Set up the plot
        fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(16,6))
        stft = np.abs(librosa.stft(y))
        spectrogram = librosa.amplitude_to_db(stft, ref=np.max)
        
        # Plot spectrogram
        librosa.display.specshow(spectrogram, sr=sr, x_axis='time', y_axis='log',cmap='coolwarm')
        plt.title('Spectrogram (dB)', fontsize = 23)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    elif (graph=="Mel Spectrogram"):
        # Set up the plot
        fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(16,6))
        mel_spec = librosa.amplitude_to_db(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000), ref=np.max)
        # Plot Mel spectrogram
        librosa.display.specshow(mel_spec, sr=sr, x_axis='time', y_axis='log', cmap='coolwarm')
        plt.title('Mel Spectrogram (dB)', fontsize = 23)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    elif (graph=="Harmonics and Percussives"):
        # Set up the plot
        fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(16,6))
        harmonic_percussive = librosa.effects.hpss(y)
        # Plot Mel spectrogram
        librosa.display.waveshow(harmonic_percussive[0], sr=sr, alpha=0.4,  label='Harmonic',color='#FFB100')
        librosa.display.waveshow(harmonic_percussive[1], sr=sr, alpha=0.4,  label='Percussive',color='#0F52BA')
        plt.legend(loc='upper right')
        plt.title('Harmonics and Percussive', fontsize = 23)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    elif (graph=="Spectral Centroids"):
        # Set up the plot
        fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(16,6))
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        # Computing the time variable for visualization
        frames = range(len(spectral_centroids))

        # Converts frame counts to time (seconds)
        t = librosa.frames_to_time(frames)
        # Plot Mel spectrogram
        librosa.display.waveshow(y, sr=sr, alpha=0.4, color='#FFB100')
        plt.plot(t, normalize(spectral_centroids), color='#0F52BA',linewidth=0.7)
        plt.title('Waveform + Spectral centroids', fontsize = 23)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    elif (graph=="Spectral Rolloffs"):
        # Set up the plot
        fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(16,6))
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        # Computing the time variable for visualization
        frames = range(len(rolloff))

        # Converts frame counts to time (seconds)
        t = librosa.frames_to_time(frames)
        # Plot Mel spectrogram
        librosa.display.waveshow(y, sr=sr, alpha=0.4, color='#FFB100')
        plt.plot(t, normalize(rolloff), color='#0F52BA',linewidth=0.7)
        plt.title('Waveform + Spectral RollOff', fontsize = 23)
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        plot_image = np.array(canvas.renderer.buffer_rgba())
        plt.close(fig)
    return plot_image, zero_crossing_rate, tempo


graph_options = ["All", "Waveform", "Short-time Fourier Transform (STFT)", "Spectrogram", "Mel Spectrogram", "Harmonics and Percussives", "Spectral Centroids", "Spectral Rolloffs"]
graph_dropdown = gr.inputs.Dropdown(graph_options, label="Select a graph to display")
im=gr.outputs.Image(type="numpy", label="Graphs")
inputs = [gr.inputs.File(label="Upload audio file (WAV format)"), graph_dropdown]
outputs = [im, gr.outputs.Label(label="Zero Crossing Rate"),gr.outputs.Label(label="Tempo")]
title = "Audio Analysis Dashboard"
description = "## Audio Feature Analysis\n\nUpload an audio file to visualize the following features:- Waveform- Short-time Fourier Transform (STFT)- Spectrogram- Mel Spectrogram- Harmonics and Percussives- Spectral Centroids- Spectral Rolloffs"
gr.Interface(fn=plot_audio_features, inputs=inputs, outputs=outputs, title=title, description=description).launch()


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:349: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `keep_filename` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [56]:
f = open('/content/drive/MyDrive/Data/genres_original/classical/classical.00035.wav', "r")
plot_audio_features(f,"Spectral Rolloffs")

(array([[[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],
 
        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],
 
        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],
 
        ...,
 
        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],
 
        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
    